In [1]:
from langchain_community.document_loaders.wikipedia import WikipediaLoader
loader = WikipediaLoader(query="MKUltra")
documents = loader.load()

c:\Users\spurusho\Downloads\GenAI\Personal\langchainTutorials\venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\spurusho\Downloads\GenAI\Personal\langchainTutorials\venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [2]:
len(documents)

24

In [3]:
len(documents[0].page_content)

4000

In [4]:
from langchain_text_splitters.character import CharacterTextSplitter
splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = splitter.split_documents(documents)

Created a chunk of size 542, which is longer than the specified 500
Created a chunk of size 525, which is longer than the specified 500


In [5]:
len(docs)

53

In [6]:
from langchain_ollama.embeddings import OllamaEmbeddings
embedding = OllamaEmbeddings(model = "llama3.2:1b")

In [9]:
from langchain_chroma.vectorstores import Chroma
db = Chroma.from_documents(docs, embedding=embedding, persist_directory='./vectorsMKUltra')

In [11]:
db._persist_directory

'./vectorsMKUltra'

In [16]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_ollama.chat_models import ChatOllama
chat = ChatOllama(model = "llama3.2:1b", temperature=0.2)
question = "When was this declassified?"
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(), llm = chat)

In [17]:
import logging
logger = logging.basicConfig(filename = "logs.txt", level = logging.INFO, format = "%(asctime)s %(level)s %(message)s")

In [22]:
unique_docs = retriever_from_llm.get_relevant_documents(question)

In [23]:
len(unique_docs)

19

In [25]:
unique_docs[0]

Document(id='30983626-e472-4509-9bae-e638a4fec500', metadata={'source': 'https://en.wikipedia.org/wiki/Project_Artichoke', 'summary': "Project Artichoke (also referred to as Operation Artichoke) was a project developed and enacted by the United States Central Intelligence Agency (CIA) for the purpose of researching methods of interrogation. Project Artichoke was succeeded by Project MKUltra, which began in 1953.\nInitially known as Project Bluebird, Project Artichoke officially arose on August 20, 1951, and was operated by the CIA's Office of Scientific Intelligence. The primary goal of Project Artichoke was to determine whether a person could be involuntarily made to perform an act of attempted assassination. The project also studied the effects of mind control and hypnosis, forced addiction to (and subsequent withdrawal from) morphine, and other chemicals, including LSD, to produce amnesia and other vulnerable states in victims.\n\n", 'title': 'Project Artichoke'}, page_content="Proj

In [26]:
db.as_retriever().get_relevant_documents(query = "When was this declassified?")

[Document(id='e6034fc2-2d03-4305-a138-a8c122106aab', metadata={'source': 'https://en.wikipedia.org/wiki/United_States_President%27s_Commission_on_CIA_Activities_within_the_United_States', 'summary': 'The United States President\'s Commission on CIA Activities within the United States was ordained by President Gerald Ford in 1975 to investigate the activities of the Central Intelligence Agency and other intelligence agencies within the United States. The Presidential Commission was led by Vice President Nelson Rockefeller, from whom it gained the nickname the Rockefeller Commission.\nThe commission was created in response to a December 1974 report in The New York Times that the CIA had conducted illegal domestic activities, including experiments on US citizens, during the 1960s. The commission issued a single report in 1975, touching upon certain CIA abuses including mail opening and surveillance of domestic dissident groups. It also publicized Project MKUltra, a CIA mind control resear

### From the above it is clear that retriever is giving 4 and multiqueryretriever gives the more docs because of more questions

User Query: "How does a transformer model work?"

🔹 Normal Retriever might just look for “transformer model” and miss related documents.

🔹 MultiQueryRetriever will also look for:

“Explain transformer architecture”

“Components of a transformer in NLP”

“Working of self-attention in transformers”

Result: More diverse and rich results.

✨ Rule of Thumb:<br>
Start with a normal retriever.<br>
If you're not getting good enough results (especially in RAG pipelines), switch to or add a MultiQueryRetriever.

